In [1]:
import pandas as pd
from typing import List, Tuple

In [2]:
def _time_to_seconds(t: str) -> float:
    """
    Konvertiert 'hh:mm:ss' (oder 'mm:ss' / 'ss') in Sekunden (float).
    """
    t = t.strip()
    if not t:
        raise ValueError("Leerer Zeit-String übergeben.")

    parts = t.split(":")
    parts = [float(p) for p in parts]
    if len(parts) == 3:
        h, m, s = parts
    elif len(parts) == 2:
        h = 0.0
        m, s = parts
    elif len(parts) == 1:
        h = 0.0
        m = 0.0
        s = parts[0]
    else:
        raise ValueError(f"Ungültiges Zeitformat: {t}")
    return h * 3600 + m * 60 + s

def label_csv_segments(
    input_csv: str,
    segments: List[Tuple[str, str, str]],
    output_csv: str
) -> tuple[pd.DataFrame, int]:
    """
    Lädt ein CSV, labelt die angegebenen Zeitbereiche (hh:mm:ss) anhand der Spalte 'timestamp'
    und speichert ein neues CSV.

    Parameters
    ----------
    input_csv : str
        Pfad zur Eingabedatei (.csv)
    segments : list of (str, str, str)
        Liste: (label, start_time, end_time) im Format 'hh:mm:ss'
    output_csv : str
        Pfad zur Ausgabe (.csv)

    Returns
    -------
    df_out : pd.DataFrame
        Gelabeltes DataFrame (mit Frame-Spalte)
    unlabeled_count : int
        Anzahl ungelabelter Frames
    """

    df = pd.read_csv(input_csv)

    if "timestamp" not in df.columns:
        raise KeyError("Spalte 'timestamp' wurde im CSV nicht gefunden.")

    # sicherstellen, dass timestamp numerisch ist (Sekunden)
    df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")

    if "Label" not in df.columns:
        df["Label"] = "None"
    else:
        df["Label"] = df["Label"].fillna("None")

    # --- Zeitsegmente anwenden ---
    for label, start_str, end_str in segments:
        start_sec = _time_to_seconds(start_str)
        end_sec   = _time_to_seconds(end_str)

        if start_sec > end_sec:
            print(f"Ungültiger Bereich {start_str}>{end_str} – übersprungen.")
            continue

        mask = (df["timestamp"] >= start_sec) & (df["timestamp"] <= end_sec)
        n_marked = int(mask.sum())

        if n_marked == 0:
            print(f"Keine Frames im Bereich {start_str}–{end_str} für Label '{label}' gefunden.")
            continue

        df.loc[mask, "Label"] = label
        first_idx = df.index[mask][0]
        last_idx  = df.index[mask][-1]
        print(
            f"Label '{label}' für Zeit {start_str}–{end_str} "
            f"(Frames {first_idx}–{last_idx}, {n_marked} Frames) gesetzt."
        )

    # --- Ungelabelte Frames zählen & Bereiche anzeigen ---
    unlabeled_mask = (df["Label"] == "None") | (df["Label"].isna())
    unlabeled_count = int(unlabeled_mask.sum())

    if unlabeled_count > 0:
        print(f"{unlabeled_count} Frames ohne Label.")

        gaps = []
        run_start = None

        for i, is_unlabeled in enumerate(unlabeled_mask.to_numpy()):
            if is_unlabeled and run_start is None:
                run_start = i
            elif not is_unlabeled and run_start is not None:
                gaps.append((run_start, i - 1))
                run_start = None

        if run_start is not None:
            gaps.append((run_start, len(df) - 1))

        if gaps:
            preview = ", ".join(f"{a}-{b}" for a, b in gaps[:5])
            if len(gaps) > 5:
                preview += f", … (+{len(gaps) - 5} weitere)"
            print(f"Ungelabelte Bereiche: {preview}")
    else:
        print("Alle Frames sind gelabelt.")

    # Frame-Index explizit als Spalte aufnehmen
    df_out = df.reset_index().rename(columns={"index": "Frame"})

    df_out.to_csv(output_csv, index=False, float_format="%.6f")
    print(f"Gespeichert: {output_csv}")

    return df_out, unlabeled_count

In [ ]:
# die run1.csv hat eine tabelle timestamp welche in Sekunden angibt zu welchem zeit punkt der frame aufgenommen wurde
# eingeben werden die zeiten aber im format hh:mm:ss
# die funktion die eingebenen zeit in sekunden umwandeln und dann alle entsprechenden frames labeln anhand der spalte timestamp

segments = [
    # (Label,           Start,      Ende)
    ("Stehen",          "00:01:21", "00:01:31"),
    ("Fahren bis Ampel","00:01:31", "00:02:00"),
    ("Stehen",          "00:02:00", "00:02:05"),
    ("Kurve",           "00:02:05", "00:02:16"),
    ("Gerade fahren",   "00:02:16", "00:02:34"),
    ("Spurwechsel",     "00:02:34", "00:02:38"),
    ("Gerade fahren",   "00:02:38", "00:02:45"),
    ("Kurve",           "00:02:45", "00:02:52"),
    ("Gerade fahren",   "00:02:52", "00:03:04"),
    ("Doppel Kurve",    "00:03:04", "00:03:22"),
    ("Gerade fahren",   "00:03:22", "00:03:40"),
    ("Spurwechsel",     "00:03:40", "00:03:45"),
    ("Gerade fahren",   "00:03:45", "00:03:50"),
    ("Kurve",           "00:03:50", "00:03:57"),
    ("Gerade fahren",   "00:03:57", "00:04:03"),
    ("Doppel Kurve",    "00:04:03", "00:04:19"),
    ("Gerade fahren",   "00:04:19", "00:04:26"),
    ("Kurve",           "00:04:26", "00:04:33"),
    ("Gerade fahren",   "00:04:33", "00:04:52"),
    ("Doppel Kurve",    "00:04:52", "00:05:05"),
    ("Kreisverkehr nähern","00:05:05", "00:05:15"),
    ("Kreisverkehr",    "00:05:15", "00:05:28"),
    ("Kurve",           "00:05:28", "00:05:34"),
    ("Gerade fahren",   "00:05:34", "00:05:49"),
    ("Kurve",           "00:05:49", "00:05:59"),
    ("Gerade fahren",   "00:05:59", "00:06:02"),
    ("Kurve",           "00:06:02", "00:06:10"),
    ("Gerade fahren",   "00:06:10", "00:06:22"),
    ("Stehen",          "00:06:22", "00:06:24"),
    ("Gerade fahren",   "00:08:26", "00:08:36"),
    ("Abbiegen",        "00:08:36", "00:08:41"),
    ("Gerade fahren",   "00:08:41", "00:08:54"),
    ("Kurve",           "00:08:54", "00:08:57"),
    ("Gerade fahren",   "00:08:57", "00:09:05"),
    ("Kreisverkehr nähern","00:09:05", "00:09:07"),
    ("Kreisverkehr",    "00:09:07", "00:09:15"),
    ("Gerade fahren",   "00:09:15", "00:09:20"),
    ("Spurwechsel",     "00:09:20", "00:09:23"),
    ("Gerade fahren",   "00:09:23", "00:09:55"),
    ("Doppel Kurve",    "00:09:55", "00:10:41"),
]


label_csv_segments(
    input_csv="run2.csv",
    segments=segments,
    output_csv="run2_labeled.csv"
)

/tmp/ipykernel_37631/1332479266.py:50: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,20,23,27,31,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,71,72,73,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_csv)


Label 'Stehen' für Zeit 00:01:21–00:01:31 (Frames 7895–8894, 1000 Frames) gesetzt.
Label 'Fahren bis Ampel' für Zeit 00:01:31–00:02:00 (Frames 8895–11794, 2900 Frames) gesetzt.
Label 'Stehen' für Zeit 00:02:00–00:02:05 (Frames 11794–12294, 501 Frames) gesetzt.
Label 'Kurve' für Zeit 00:02:05–00:02:16 (Frames 12295–13394, 1100 Frames) gesetzt.
Label 'Gerade fahren' für Zeit 00:02:16–00:02:34 (Frames 13395–15194, 1800 Frames) gesetzt.
Label 'Spurwechsel' für Zeit 00:02:34–00:02:38 (Frames 15195–15594, 400 Frames) gesetzt.
Label 'Gerade fahren' für Zeit 00:02:38–00:02:45 (Frames 15595–16294, 700 Frames) gesetzt.
Label 'Kurve' für Zeit 00:02:45–00:02:52 (Frames 16295–16994, 700 Frames) gesetzt.
Label 'Gerade fahren' für Zeit 00:02:52–00:03:04 (Frames 16995–18193, 1199 Frames) gesetzt.
Label 'Doppel Kurve' für Zeit 00:03:04–00:03:22 (Frames 18194–19994, 1801 Frames) gesetzt.
Label 'Gerade fahren' für Zeit 00:03:22–00:03:40 (Frames 19995–21793, 1799 Frames) gesetzt.
Label 'Spurwechsel' für Z

(       Frame throttle brakes cruiseControl{1}.mask_objects_adas  \
 0          0        -      -                                  -   
 1          1        0      0                                  0   
 2          2        0      0                                  0   
 3          3        0      0                                  0   
 4          4  0.00066      0                                  0   
 ...      ...      ...    ...                                ...   
 74201  74201      0.0    0.0                                  0   
 74202  74202      0.0    0.0                                  0   
 74203  74203      0.0    0.0                                  0   
 74204  74204      0.0    0.0                                  0   
 74205  74205      0.0    0.0                                  0   
 
       car0{1}.shift_up car0{1}.shift_down wheel_position  \
 0                    -                  -              -   
 1                    0                  0              0   